In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from kiblib.utils.db import DbConn

In [2]:
db_conn = DbConn().create_engine()

In [3]:
# On importe le fichier rempli par Mathilde M. contenant les données du dernier semestre
action_culturelle = pd.read_excel("/home/kibini/FréquentationAC_2024_semestre1.xlsx",sheet_name="Action culturelle")
action_culturelle

,Evénement,Date,Porteur de projet,Animation,Lieu Action,Type d'action,Partenariat,public,Nombre de personnes venues,Notes,Jauge,Saturation de l'espace
0,NaN,2024-01-03,JB,Jeux vidéo,La criée,Jeux,NaN,Tout public,60.0,NaN,NaN,NaN
1,NaN,2024-01-06,JB,Jeux vidéo,La criée,Jeux,NaN,Tout public,55.0,NaN,NaN,NaN
2,NaN,2024-01-13,Christine,39-45 : elles n'ont rien oublié,La criée,Projection/diffusion,NaN,Adultes,80.0,NaN,45.0,NaN
3,NaN,2024-01-14,Christine,Tricot des Lulus,La criée,Atelier,NaN,Tout public,22.0,NaN,20.0,NaN
4,NaN,2024-01-16,Maïté,Atelier informatique pour débutants,Espace multimédia,Atelier,NaN,Adultes,10.0,NaN,12.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
125,NaN,2024-07-12,Bruno,La philo en herbe,Hors les murs,Atelier,NaN,Tout public,NaN,NaN,NaN,NaN
126,NaN,2024-07-20,Mathilde M.,Ceci est mon Non,La criée,Spectacle,Les Plaines d'été,Tout public,17.0,NaN,NaN,NaN
127,NaN,2024-07-20,Mathilde M.,Mogrr,La criée,Spectacle,Les Plaines d'été,Tout public,25.0,NaN,NaN,NaN
128,NaN,2024-08-31,Julie,Ma lessive Zéro Déchet,La criée,Atelier,NaN,Adultes,24.0,NaN,NaN,NaN


In [4]:
action_culturelle.columns

Index(['Evénement', 'Date', 'Porteur de projet', 'Animation', 'Lieu Action',
       'Type d'action', 'Partenariat', 'public', 'Nombre de personnes venues',
       'Notes', 'Jauge', 'Saturation de l'espace'],
      dtype='object')

In [5]:
# On renomme les colonnes pour qu'elles correspondent aux colonnes dans stadb.action_culturelle
action_culturelle = action_culturelle.rename(columns={"Evénement":"evenement",
                                                      "Date":"date",
                                                      "Animation":"action",
                                                      "Lieu Action":"lieu",
                                                      "Type d'action":"type",
                                                      "Partenariat":"partenariat",
                                                      "public":"public",
                                                      "Nombre de personnes venues":"participants",
                                                      "Notes":"notes",
                                                      "Jauge":"jauge"})

In [6]:
action_culturelle.dtypes

evenement                         object
date                      datetime64[ns]
Porteur de projet                 object
action                            object
lieu                              object
type                              object
partenariat                       object
public                            object
participants                     float64
notes                             object
jauge                            float64
Saturation de l'espace           float64
dtype: object

In [7]:
action_culturelle['date'] = pd.to_datetime(action_culturelle['date']).dt.date

In [8]:
action_culturelle.dtypes

evenement                  object
date                       object
Porteur de projet          object
action                     object
lieu                       object
type                       object
partenariat                object
public                     object
participants              float64
notes                      object
jauge                     float64
Saturation de l'espace    float64
dtype: object

In [9]:
action_culturelle = action_culturelle.drop(labels=["Porteur de projet","Saturation de l'espace"],axis=1)

In [10]:
# On remet les colonnes dans le bon ordre
action_culturelle = action_culturelle[["date","action","lieu","type","public","partenariat","participants","jauge","evenement","notes"]]

action_culturelle

,date,action,lieu,type,public,partenariat,participants,jauge,evenement,notes
0,2024-01-03,Jeux vidéo,La criée,Jeux,Tout public,NaN,60.0,NaN,NaN,NaN
1,2024-01-06,Jeux vidéo,La criée,Jeux,Tout public,NaN,55.0,NaN,NaN,NaN
2,2024-01-13,39-45 : elles n'ont rien oublié,La criée,Projection/diffusion,Adultes,NaN,80.0,45.0,NaN,NaN
3,2024-01-14,Tricot des Lulus,La criée,Atelier,Tout public,NaN,22.0,20.0,NaN,NaN
4,2024-01-16,Atelier informatique pour débutants,Espace multimédia,Atelier,Adultes,NaN,10.0,12.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
125,2024-07-12,La philo en herbe,Hors les murs,Atelier,Tout public,NaN,NaN,NaN,NaN,NaN
126,2024-07-20,Ceci est mon Non,La criée,Spectacle,Tout public,Les Plaines d'été,17.0,NaN,NaN,NaN
127,2024-07-20,Mogrr,La criée,Spectacle,Tout public,Les Plaines d'été,25.0,NaN,NaN,NaN
128,2024-08-31,Ma lessive Zéro Déchet,La criée,Atelier,Adultes,NaN,24.0,NaN,NaN,NaN


# MODIFICATION DES DONNÉES DANS LA BASE

In [11]:
#action_culturelle.to_sql(name="stat_action_culturelle",con=db_conn,if_exists='append',index=False)

130